In [11]:
import QuantLib as ql
import pandas as pd
from scipy.optimize import minimize

# Market data
spot_price = 176.65
strike_price = 190
market_price = 0.17  # Example market price of the option
risk_free_rate = 0.0525
dividend_yield = 0.0054
traditional_implied_volatility = 0.189461
calculation_date = ql.Date(3, 11, 2023)
maturity_date = ql.Date(24, 11, 2023)

def HestonParameters(spot_price, strike_price, market_price, dividend_yield, traditional_implied_volatility, calculation_date, maturity_date, risk_free_rate = .00525):
    day_count = ql.Actual365Fixed()
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

    # Set up the QuantLib environment
    ql.Settings.instance().evaluationDate = calculation_date
    option_type = ql.Option.Call  # or ql.Option.Put for a put option
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # Initial parameters for the Heston model
    initial_v0 = 0.1
    initial_kappa = 0.1
    initial_theta = traditional_implied_volatility
    initial_sigma = 0.1
    initial_rho = 0.1

    # Define the optimization objective function
    def objective_function(params):
        v0, kappa, theta, sigma, rho = params

        heston_process = ql.HestonProcess(
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count)),
            ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_yield, day_count)),
            ql.QuoteHandle(ql.SimpleQuote(spot_price)),
            v0, kappa, theta, sigma, rho
        )

        model = ql.HestonModel(heston_process)
        engine = ql.AnalyticHestonEngine(model)
        european_option.setPricingEngine(engine)

        model_price = european_option.NPV()
        error = (model_price - market_price) ** 2  # Squared error
        return error, model_price  # Return both error and the model price

    # Bounds for the parameters, excluding theta which is input by the user
    bounds = [(0.0001, 1.0), (0.0001, 2.0), (traditional_implied_volatility * 0.5, traditional_implied_volatility * 1.5), (0.0001, 1.0), (-0.999, 0.999)]

    # Initial parameter guesses, excluding theta which is input by the user
    initial_guess = [0.1, 0.1, traditional_implied_volatility, 0.1, 0.1]

    # Dictionary of optimizers to test, removing poor performers
    optimizers = {
        'L-BFGS-B': {'method': 'L-BFGS-B'},
        'TNC': {'method': 'TNC'},
    }

    # DataFrame to collect results
    results_df = pd.DataFrame(columns=['Optimizer', 'Success', 'Params', 'Objective_Value', 'Estimated_Price', 'Market_Price', 'MSE'])

    # Run optimizers and collect results
    for name, opt in optimizers.items():
        print(f"Running optimizer: {name}")
        
        # Run the optimizer and capture the additional returned value
        result = minimize(lambda x: objective_function(x)[0], initial_guess, method=opt['method'], bounds=bounds)
        
        success = result.success
        params = result.x if success else None
        # Now retrieve the full result from the objective function which includes the error and the model price
        objective_value, estimated_price = objective_function(params) if success else (None, None)
        error = (estimated_price - market_price) if success else None
        
        # Create a temporary DataFrame and concatenate it to the main DataFrame
        temp_df = pd.DataFrame({
            'Optimizer': [name],
            'Success': [success],
            'Params': [params],
            'Objective_Value': [objective_value],
            'Estimated_Price': [estimated_price],
            'Market_Price': [market_price if success else None],
            'MSE': [error**2]
        })
        results_df = pd.concat([results_df, temp_df], ignore_index=True)

    # Print the DataFrame
    results_df.sort_values(by='MSE', ascending= True)

    


In [2]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Heston model parameters
S0 = 100       # Initial stock price
v0 = 0.04      # Initial variance
r = 0.05       # Risk-free rate
T = 1.0        # Time horizon
dt = 0.01      # Time step
N = int(T / dt) # Number of time steps
time = np.linspace(0, T, N)

# Generate correlated Brownian motions
dw1 = np.random.normal(size=N-1, scale=np.sqrt(dt))

# Function to generate variance paths for given Heston model parameters
def generate_variance_paths(kappa, theta, sigma, rho):
    v_temp = np.zeros(N)
    v_temp[0] = v0
    dw2 = rho * dw1 + np.sqrt(1 - rho**2) * np.random.normal(size=N-1, scale=np.sqrt(dt))

    for t in range(1, N):
        v_temp[t] = max(v_temp[t-1] + kappa * (theta - max(v_temp[t-1], 0)) * dt + sigma * np.sqrt(max(v_temp[t-1], 0)) * dw2[t-1], 0)

    return v_temp

# Initial parameters for the sliders
initial_kappa = 1.0
initial_theta = 0.04
initial_sigma = 0.2
initial_rho = -0.7

# Create the figure
fig = make_subplots(rows=1, cols=1)

# Add trace for initial parameters
fig.add_trace(
    go.Scatter(
        x=time, 
        y=generate_variance_paths(initial_kappa, initial_theta, initial_sigma, initial_rho),
        mode='lines'
    ),
    row=1, col=1
)

# Update the layout
fig.update_layout(
    title='Sensitivity Analysis of Variance Path with Multiple Sliders',
    xaxis_title='Time',
    yaxis_title='Variance v(t)',
    sliders=[
        {
            'pad': {'t': 30},
            'steps': [
                {
                    'method': 'update',
                    'args': [{'y': [generate_variance_paths(kappa, initial_theta, initial_sigma, initial_rho)]}],
                    'label': f'{kappa:.2f}'
                } for kappa in np.arange(0.1, 2.0, 0.1)
            ],
            'currentvalue': {'prefix': 'Mean reversion rate (kappa): '}
        },
        {
            'pad': {'t': 60},
            'steps': [
                {
                    'method': 'update',
                    'args': [{'y': [generate_variance_paths(initial_kappa, theta, initial_sigma, initial_rho)]}],
                    'label': f'{theta:.2f}'
                } for theta in np.arange(0.01, 0.1, 0.01)
            ],
            'currentvalue': {'prefix': 'Long-term mean variance (theta): '}
        },
        {
            'pad': {'t': 90},
            'steps': [
                {
                    'method': 'update',
                    'args': [{'y': [generate_variance_paths(initial_kappa, initial_theta, sigma, initial_rho)]}],
                    'label': f'{sigma:.2f}'
                } for sigma in np.arange(0.1, 0.5, 0.05)
            ],
            'currentvalue': {'prefix': 'Volatility of volatility (sigma): '}
        },
        {
            'pad': {'t': 120},
            'steps': [
                {
                    'method': 'update',
                    'args': [{'y': [generate_variance_paths(initial_kappa, initial_theta, initial_sigma, rho)]}],
                    'label': f'{rho:.2f}'
                } for rho in np.arange(-0.9, 1.0, 0.1)
            ],
            'currentvalue': {'prefix': 'Correlation (rho): '}
        }
    ],
    autosize=False,
    width=800,
    height=600
)

# Show the figure in a Jupyter Notebook
fig.show()

# If you are not using Jupyter, you might use this instead:
# plotly.offline.plot(fig, filename='heston_model_sensitivity_multiple_sliders.html')
